# Korean LLM (Large Language Model) Serving on SageMaker with AWS Large Model Container DLC
---

한국어 LLM 모델 SageMaker 서빙 핸즈온 (No inference code)

- LLM GitHub: https://github.com/nlpai-lab/KULLM
- Hugging Face model hub: https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2
- [AWS Blog: Deploy large models on Amazon SageMaker using DJLServing and DeepSpeed model parallel inference](https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference)

In [4]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 

In [5]:
import os
import sagemaker, boto3, jinja2
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment()  # jinja environment to generate model configuration templates

<br>

## 1. Download LLM model and upload it to S3
---

In [6]:
from huggingface_hub import snapshot_download
from pathlib import Path

model_id = "nlpai-lab/kullm-polyglot-12.8b-v2"
model_prefix = model_id.split('/')[-1].replace('.', '-')
model_tar_dir = f"/home/ec2-user/SageMaker/models/{model_prefix}"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

if not os.path.isdir(model_tar_dir):
    os.makedirs(model_tar_dir, exist_ok=True)
    # - Leverage the snapshot library to donload the model since the model is stored in repository using LFS    
    snapshot_download(
        repo_id=model_id,
        local_dir=str(model_tar_dir), 
        local_dir_use_symlinks=False,        
        allow_patterns=allow_patterns,
        cache_dir="/home/ec2-user/SageMaker/"        
    )

bucket_prefix = 'ko-llms/serving'    
s3_code_prefix = f"{bucket_prefix}/{model_prefix}/code"  # folder within bucket where code artifact will go
s3_model_prefix = f"{bucket_prefix}/{model_prefix}/model"  # folder where model checkpoint will go
s3_model_artifact = f"s3://{bucket}/{s3_model_prefix}"

print(f"S3 code prefix \n {s3_code_prefix}")
print(f"S3 model prefix: \n {s3_model_prefix}")
print(f"S3 model artifact path: \n {s3_model_artifact}")

S3 code prefix 
 ko-llms/serving/kullm-polyglot-12-8b-v2/code
S3 model prefix: 
 ko-llms/serving/kullm-polyglot-12-8b-v2/model
S3 model artifact path: 
 s3://sagemaker-us-west-2-339967300945/ko-llms/serving/kullm-polyglot-12-8b-v2/model


In [7]:
%%bash
aws configure set default.s3.max_concurrent_requests 100
aws configure set default.s3.max_queue_size 10000
aws configure set default.s3.multipart_threshold 1GB
aws configure set default.s3.multipart_chunksize 64MB

In [8]:
!aws s3 sync {model_tar_dir} {s3_model_artifact}
print(f"Model uploaded to --- > {s3_model_artifact}")
print(f"We will set option.s3url={s3_model_artifact}")

Model uploaded to --- > s3://sagemaker-us-west-2-339967300945/ko-llms/serving/kullm-polyglot-12-8b-v2/model
We will set option.s3url=s3://sagemaker-us-west-2-339967300945/ko-llms/serving/kullm-polyglot-12-8b-v2/model


<br>

## 2. Model Serving Scripts
---
### Create `serving.properties`

이 설정 파일은 어떤 추론 최적화 라이브러리를 사용할지, 어떤 설정을 사용할지 DJL Serving에 알려주는 설정 파일입니다. 필요에 따라 적절한 구성을 설정할 수 있습니다.

There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the **FasterTransformer**
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. `djl_python.fastertransformer` refers to the `fastertransformer.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url`: Set this to the URI of the Amazon S3 bucket that contains the model. When this is set, the container leverages [s5cmd](https://github.com/peak/s5cmd) to download the model from s3. This enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.tensor_parallel_degree`: Set to the number of GPU devices over which DeepSpeed needs to partition the model. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests.

`serving.properties`의 일반적인 설정법과 자세한 내용은 https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html 를 참조하세요.

<img src="./images/Slide21.png" width="800"/>

In [9]:
src_path = f"src/{model_prefix}"
!rm -rf {src_path}
os.makedirs(src_path, exist_ok=True)

In [10]:
%%writefile {src_path}/serving.properties
engine = FasterTransformer
option.entryPoint = djl_python.fastertransformer
option.s3url = {{s3url}}
option.tensor_parallel_degree = 4
option.dtype = fp16
option.task = text-generation

Writing src/kullm-polyglot-12-8b-v2/serving.properties


### serving.properties의 S3 경로 수정

In [11]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path(f"{src_path}/serving.properties").open().read())
Path(f"{src_path}/serving.properties").open("w").write(template.render(s3url=s3_model_artifact))
!pygmentize {src_path}/serving.properties | cat -n

     1	engine = FasterTransformer
     2	option.entryPoint = djl_python.fastertransformer
     3	option.s3url = s3://sagemaker-us-west-2-339967300945/ko-llms/serving/kullm-polyglot-12-8b-v2/model
     4	option.tensor_parallel_degree = 4
     5	option.dtype = fp16
     6	option.task = text-generation


### Create the Tarball and then upload to S3

In [12]:
!rm -rf model.tar.gz
!tar czvf model.tar.gz -C {src_path} .

./
./serving.properties


In [13]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")
#!rm -rf model.tar.gz

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-339967300945/ko-llms/serving/kullm-polyglot-12-8b-v2/code/model.tar.gz


<br>

## 3. Serve LLM Model on SageMaker

---

### Create SageMaker Model

SageMaker 엔드포인트 생성 매개변수 VolumeSizeInGB를 지정할 때 마운트되는 Amazon EBS(Amazon Elastic Block Store) 볼륨에 /tmp를 매핑하기 때문에 컨테이너는 인스턴스의 `/tmp` 공간에 모델을 다운로드합니다. 이때 s5cmd (https://github.com/peak/s5cmd) 를 활용하므로 대용량 모델을 빠르게 다운로드할 수 있습니다.
볼륨 인스턴스와 함께 미리 빌드되어 제공되는 p4dn과 같은 인스턴스의 경우 컨테이너의 `/tmp`를 계속 활용할 수 있습니다. 

In [14]:
from sagemaker.utils import name_from_base
from sagemaker import image_uris

img_uri = image_uris.retrieve(framework="djl-fastertransformer", region=region, version="0.23.0")
model_name = name_from_base(f"{model_prefix}")
print(f"Image going to be used is ---- > {img_uri}")
print(model_name)

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-fastertransformer5.3.0-cu118
kullm-polyglot-12-8b-v2-2023-09-06-04-40-28-722


In [15]:
from sagemaker import Model, image_uris, serializers, deserializers
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")
env = {"HUGGINGFACE_HUB_CACHE": "/tmp", "TRANSFORMERS_CACHE": "/tmp"}

model = Model(image_uri=img_uri, model_data=s3_code_artifact, env=env, role=role)

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-339967300945/ko-llms/serving/kullm-polyglot-12-8b-v2/code/model.tar.gz


### Create SageMaker Endpoint

In [16]:
from sagemaker.utils import name_from_base

instance_type = "ml.g5.12xlarge"
endpoint_name = name_from_base(f"{model_prefix}-djl-ft")
print(endpoint_name)

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=3600,
    wait=False
)

kullm-polyglot-12-8b-v2-djl-ft-2023-09-06-04-40-45-340


엔드포인트가 생성되는 동안 아래의 문서를 같이 확인해 보세요.
- https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html

In [17]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [31]:
endpoint_name


'kullm-polyglot-12-8b-v2-djl-ft-2023-09-06-04-40-45-340'

In [24]:
%%time 
from inference_lib import describe_endpoint, Prompter
describe_endpoint(endpoint_name)         

Endpoint is  InService
CPU times: user 20.6 ms, sys: 4.41 ms, total: 25 ms
Wall time: 195 ms


<br>

## 4. Inference
---

In [28]:
import json
from inference_lib import Prompter
prompter = Prompter("kullm")


In [22]:
import json
from inference_lib import KoLLMSageMakerEndpoint

ep = KoLLMSageMakerEndpoint(endpoint_name)

In [29]:
params = {
    "do_sample": False,
    "max_new_tokens": 256,
    "return_full_text": True,
    "temperature": 0.01,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}


instruction = "아래 질문에 100글자 이상으로 자세하게 대답해줘."
#instruction = ""
input_text = "고려대학교에 대해서 알려줘"
prompt = prompter.generate_prompt(instruction, input_text)
payload = {
    "inputs": [prompt,],
    "parameters": params
}

In [39]:
%%timeit -n3 -r1

payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

('Response: AWS는 Amazon Web Services의 약자입니다. 클라우드 컴퓨팅 서비스를 제공하는 선도 기술 회사입니다. '
 'AWS는 전 세계 고객에게 인프라, 애플리케이션 및 서비스를 제공하는 데 사용되는 클라우드 컴퓨팅 플랫폼입니다. AWS는 고객이 클라우드 '
 '컴퓨팅을 통해 애플리케이션와 애플리케이션을 관리하고 확장할 수 있도록 지원하. AWS는 고객mazon Web Services의 클라우드 '
 '컴퓨팅 플랫폼을 통해 고객이 컴퓨팅, 스토리지, 네트워킹, 데이터베이스, 보안 등 다양한 서비스를 사용할 수 있는 Amazon Web '
 'Services의 클라우드 컴퓨팅 플랫폼을 통해 고객이 컴퓨팅, 스토리지, 네트워킹, 데이터베이스, 보안 등 다양한 서비스를 사용할 수 '
 '있습니다. AWS는 고객이 클라우드 컴퓨팅을 통해 데이터를 저장하고 관리할 수 있도록 지원합니다. AWS는 고객이 클라우드 컴퓨팅을 통해 '
 '데이터를 저장하고 관리할 수 있도록 지원합니다. AWS는 고객이 클라우드 컴퓨팅을 통해 데이터를 저장하고 관리할 수 있도록 지원합니다. '
 'AWS는 고객이 클라우드 컴퓨팅을 통해 데이터를 저장하고 관리할 수 있도록 지원합니다. AWS는 고객이 클라우드 컴퓨팅을 통해 데이터를 '
 '저장하고 관리할 수 있도록 지원합니다. AWS는 고객이 클라우드 컴퓨팅을 통해 데이터를 저장하고 관리할 수 있도록 '
 '지원합니다.<|endoftext|>[이투데이 최환 기자]["경제 경제 회복에 대한 신뢰감 회복"] 미국의 주간 신규 실업수당 청구건수가 '
 '예상보다 큰 폭으로 감소한 것으로 나타났다. 미 노동부는 지난주 신규 실업수당 청구건수가 전주대비 7000만건 감소한 47만8000건을 '
 '기록했다고 밝혔다. 이는 전문가 예상치인 47만건건을 상회하는 수치다. 그러나 이를 나타내는 4주 평균 청구건수는 47만6750건으로 '
 '전주대비 2000건 증가했다. 이는 또 연속수당 청구건수가 전주대비 3000건 감소한 47만

In [32]:
# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

In [33]:
%%time
predictor.predict(
    {
        "inputs": ["아마존 웹 서비스에 대해서 자세히 알려줘"],
        "parameters": {"max_seq_len": 200, "temperature": 0.1},
    }
)

CPU times: user 17.5 ms, sys: 0 ns, total: 17.5 ms
Wall time: 3.42 s


[{'generated_text': '!A: 아마존 웹 서비스(AWS)는 클라우드 컴퓨팅 서비스를 제공하는 회사입니다. 클라우드 컴퓨팅은 인터넷을 통해 컴퓨팅 리소스를 제공하는 것을 말합니다. AWS는 컴퓨팅 리소스를 제공하고, 이를 통해 기업이 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원합니다.AWS는 전 세계에서 가장 큰 클라우드 컴퓨팅 서비스 제공업체 중 하나이며, 전 세계에서 가장 큰 클라우드 컴퓨팅 회사입니다.AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원하는 서비스를 제공합니다. AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원하는 서비스를 제공합니다.AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원하는 서비스를 제공합니다.AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할'}]

In [34]:
params = {
    "max_seq_len": 512,
    "temperature": 0.2,
    "top_k": 0,
    "top_p": 0.9,
}

instruction = ""
input_text = "아마존 웹 서비스에 대해서 자세히 알려줘"
payload = ep.get_payload(instruction, input_text, params)

In [35]:
%%time
predictor.predict(payload)

TypeError: Object of type bytes is not JSON serializable

In [23]:
%store endpoint_name

Stored 'endpoint_name' (str)


<br>

## 5. Clean Up
---

In [24]:
!rm -rf src

In [40]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

<br>

# References
---

- Model 정보
    - kullm-polyglot-5.8b-v2
        - This model is a parameter-efficient fine-tuned version of EleutherAI/polyglot-ko-5.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2        
    - kullm-polyglot-12.8b-v2
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b      
- 코드
    - [Boto3](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/pytorch_deploy_large_GPT_model/GPT-J-6B-model-parallel-inference-DJL.ipynb)
    - [Python SDK](https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb)
    - [Kor LLM on SageMaker](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker)
    - [AWS Generative AI Workshop for Korean language](https://github.com/aws-samples/aws-ai-ml-workshop-kr/tree/master/genai)